In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df1 = pd.read_excel('co_data.xlsx', sheet_name='Neural_network_forecasts')

In [4]:
df2 = df1[['Corn','Soy','Wheat']]
df2_1 = df1[['Barley','Mixed O/B','Oats']]

#For Barley, Oats and mixed which has less observations
df3 = df2_1.iloc[61:169]

#Each crop is modeled separately
value1 = df3.Oats

# Generate date range
dates = pd.date_range(start='2015-08-01', end='2024-07-01', freq='MS')
df = pd.DataFrame({'value': value1.values}, index=dates)

In [49]:
# -------------------------------
# 2. Normalize data
# -------------------------------
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df[['value']])

# -------------------------------
# 3. Create sequences for LSTM
# -------------------------------
def create_sequences(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

window_size = 12  # 12 months
X, y = create_sequences(scaled_data, window_size)

# Reshape input to [samples, time steps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# -------------------------------
# 4. Train/test split
# -------------------------------
train_size = len(X) - 12  # Leave last 12 for forecasting
X_train, y_train = X[:train_size], y[:train_size]

# -------------------------------
# 5. Build and train LSTM model
# -------------------------------
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(window_size, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

early_stop = EarlyStopping(monitor='loss', patience=25, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=300, verbose=1, callbacks=[early_stop])

Epoch 1/300


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 0.0475
Epoch 2/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0375
Epoch 3/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0367
Epoch 4/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0328
Epoch 5/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0310
Epoch 6/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0285
Epoch 7/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0323
Epoch 8/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.0283
Epoch 9/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.0313
Epoch 10/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0296
Epoch 11/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0324
Epoch 12/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0247
Epoch 13/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0330 
Epoch 14/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0251
Epoch 15/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0301
Epoch 16/300
3/3 ━━━━━━━━━━━━━

In [ ]:
# -------------------------------
# 6. Forecast next 12 months
# -------------------------------
# Start from the last window of data
last_sequence = scaled_data[-window_size:]
forecast_input = last_sequence.reshape((1, window_size, 1))

forecast = []
for _ in range(12):
    next_pred = model.predict(forecast_input)[0][0]
    forecast.append(next_pred)
    # Update the input sequence
    forecast_input = np.append(forecast_input[:, 1:, :], [[[next_pred]]], axis=1)

# Inverse transform to get actual forecast values
forecast_actual = scaler.inverse_transform(np.array(forecast).reshape(-1, 1)).flatten()

# -------------------------------
# 7. Build forecast date index
# -------------------------------
future_dates = pd.date_range(start=df.index[-1] + pd.DateOffset(months=1), periods=12, freq='MS')
forecast_df = pd.DataFrame({'Forecast': forecast_actual}, index=future_dates)

# -------------------------------
# 8. Plot results
# -------------------------------
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['value'], label='Observed')
plt.plot(forecast_df.index, forecast_df['Forecast'], label='LSTM Forecast', linestyle='--')
plt.title('LSTM Forecast for Next 12 Months')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# -------------------------------
# 9. Output forecast as table
# -------------------------------
print(forecast_df)

In [ ]:
# -------------------------------
# Run all columns together - in a loop
# -------------------------------

# Generate date range; adjust based on crop
dates = pd.date_range(start='2010-07-01', end='2025-03-01', freq='MS')

In [ ]:
# -------------------------------
# Assume df has 6 columns with datetime index
# -------------------------------
# Example structure:
# df.columns = ['col1', 'col2', ..., 'col6']
# df.index is monthly from 2010-07 to 2025-03

window_size = 12
forecast_horizon = 12

# Store all forecasts
all_forecasts = {}

# Loop through each column
for col in df2.columns:
    print(f"Training on {col}...")

    # 1. Normalize
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df2[[col]])

    # 2. Create sequences
    def create_sequences(data, window_size):
        X, y = [], []
        for i in range(len(data) - window_size):
            X.append(data[i:i + window_size])
            y.append(data[i + window_size])
        return np.array(X), np.array(y)

    X, y = create_sequences(scaled_data, window_size)
    X = X.reshape((X.shape[0], window_size, 1))

    # 3. Train/test split
    X_train, y_train = X[:-forecast_horizon], y[:-forecast_horizon]

    # 4. Define and train model
    model = Sequential()
    model.add(LSTM(64, activation='relu', input_shape=(window_size, 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    early_stop = EarlyStopping(monitor='loss', patience=25, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=400, verbose=0, callbacks=[early_stop])

    # 5. Forecast
    last_seq = scaled_data[-window_size:]
    forecast_input = last_seq.reshape((1, window_size, 1))

    forecast_scaled = []
    for _ in range(forecast_horizon):
        next_pred = model.predict(forecast_input, verbose=0)[0][0]
        forecast_scaled.append(next_pred)
        forecast_input = np.append(forecast_input[:, 1:, :], [[[next_pred]]], axis=1)

    forecast = scaler.inverse_transform(np.array(forecast_scaled).reshape(-1, 1)).flatten()

    # 6. Create forecast DataFrame
    last_date = df2.index[-1]
    future_dates = pd.date_range(start=last_date + pd.DateOffset(months=1), periods=forecast_horizon, freq='MS')
    forecast_df = pd.DataFrame({f'{col}_forecast': forecast}, index=future_dates)
    all_forecasts[col] = forecast_df

    # 7. Plot
    plt.figure(figsize=(12, 4))
    plt.plot(df2.index, df2[col], label='Observed')
    plt.plot(forecast_df.index, forecast_df[f'{col}_forecast'], label='Forecast', linestyle='--', color='orange')
    plt.title(f'LSTM Forecast for {col}')
    plt.xlabel('Date')
    plt.ylabel(col)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# -------------------------------
# Combine forecasts into one DataFrame
# -------------------------------
forecast_combined = pd.concat(all_forecasts.values(), axis=1)
print(forecast_combined)


In [ ]:
forecast_combined.sum(axis = 0)

,0
Corn_forecast,7921853.50
Soy_forecast,3621851.25
Wheat_forecast,2134047.75
